In [ ]:
import pandas as pd
import numpy as np

## Historische Emissionen

Die zur Zeit aktuellsten Daten zu Treibhaus finden sich im Leipziger Amtsblatt Nr. 20/2019 (1), Seite 3. Die Daten sind als Tonnen pro Kopf angegeben. 

[1] https://static.leipzig.de/fileadmin/mediendatenbank/leipzig-de/Stadt/01.1_Geschaeftsbereich_OBM/12_Ref_Kommunikation/Amtsblatt/2019/Amtsblatt_2019-20.pdf

In [ ]:
data = [
    (2011, 2012, 2013, 2014, 2015, 2016, 'Bereiche'),
    (6.42, 6.45, 6.63, 6.21, 6.15, 5.96, 'Gesamt'),
    (3.13, 3.05, 3.21, 3.04, 2.93, 2.79, 'Wirtschaft und kommunale Einrichtungen'),
    (1.90, 2.02, 2.06, 1.79, 1.83, 1.81, 'Haushalte'),
    (1.39, 1.37, 1.37, 1.38, 1.39, 1.36, 'Verkehr')
]

df = pd.DataFrame(data[1:], columns=data[0])
df

Um dem Format von Klimawatch zu entsprechen wird die Tabelle von einem "wide" Format in ein "long" format gewandelt, die Spalten umbenannt, sowie Typ und Quelle hinzugefügt. Die Emissionen werden in 1000t CO2 (kT) umgerechnet.

In [ ]:
# convert to 'long' format
emissions = df.melt(id_vars=['Bereiche'], var_name='year', value_name='per_capita')

# add missing columns
emissions["type"] = "real"
emissions["note"] = "Amtsblatt Nr. 20/2019"

# rename category, use required value for total emissions and mark last emissions data entry
emissions.rename(columns={'Bereiche': 'category'}, inplace=True)
emissions.loc[(emissions['category'] == 'Gesamt') & (emissions['year'] == 2016), 'note'] = 'last_emissions'

# use year column as index
emissions.set_index('year', inplace=True)
emissions.index = emissions.index.astype(int)

emissions

Zur Berechnung der Emissionen in absoluten Werten benötigen wir die entsprechenden Bevölkerungszahlen. Die gibt es beim Statistikamt Leipzig (1).

[1] https://statistik.leipzig.de/statcity/table.aspx?cat=2&rub=4&per=y

In [ ]:
# Bevölkerung mit Hauptwohnsitz historisch
population = pd.read_csv('./leipzig-population-historic.csv', nrows=1)
population = population.loc[0, "2011":]

population = pd.DataFrame.from_dict({
    "year": population.index.values.astype(int),
    "value": population.values,
    "type": "real"
})

population.set_index('year', inplace=True)
population

Damit lassen sich jetzt die Emissionswerte in absoluten Zahlen berechnen. Da Klimawatch mit 1000t (kt) CO2 arbeitet, rechnen wir das im selben Schritt um und runden auf zwei Nachkommastellen.

In [ ]:
emissions['value'] = emissions.loc[2011:2016, 'per_capita'] * population.loc[2011:2016, 'value'] / 1000
emissions['value'] = emissions.value.astype(float).round(2)
emissions

## Bevölkerung

Die Emissionsziele in Leipzig sind als angestrebte Pro-Kopf Emissionen definiert. Um die absoluten Emissionen zu erhalten brauchen wir dafür die historischen (1) und prognostizierten (2) Bevölkerungszahlen. Da die Bevölkerungsvorausschätzung nur Daten für Einwohner mit Hauptwohnsitz enthält wählen wir auch diese für die historischen Daten. Die Tabelle aus der Voraussschätzung (S.24) wurde wieder mit Tabula extrahiert. 

[1] https://statistik.leipzig.de/statcity/table.aspx?cat=2&rub=4&per=y

[2] https://www.leipzig.de/fileadmin/mediendatenbank/leipzig-de/Stadt/02.1_Dez1_Allgemeine_Verwaltung/12_Statistik_und_Wahlen/Stadtforschung/Bevolkerungsvorausschatzung_2019.pdf

In [ ]:
parse_thousands = lambda x: (int(float(x.replace(",", ".")) * 1000 ))

population_forecast = pd.read_csv(
    './leipzig-population-forecast-2019.csv', 
    header=None,
    usecols=[0,8],
    names=["year", "value"],
    converters={"value": parse_thousands},
    index_col=['year']
)
population_forecast['type'] = 'geplant'

population = population.append(population_forecast.loc[2019:])
population

## Emissionsziele

Die Emissionsziele sind im "Energie- und Klimaschutzkonzept der Stadt Leipzig 2014-2020" (1), das im Mai 2014 beschlossen (2) wurde. Auf Seite 17 findet sich Abbildung 7, in der die Reduktionszielwerte (in t CO2 / Kopf) für 2020 bis 2050 dargestellt sind. Kombiniert mit der prognostizierten Bevölkerungsentwicklung lassen sich die Gesamtemissionsziele in absoluten Zahlen errechnen - mit Ausnahme von 2050, da die Bevölkerungsschätzung nicht soweit reicht.

[1] https://static.leipzig.de/fileadmin/mediendatenbank/leipzig-de/Stadt/02.3_Dez3_Umwelt_Ordnung_Sport/36_Amt_fuer_Umweltschutz/Energie_und_Klima/Klimaschutz/Energie-und_Klimaschutzprogramm_2014-2020.pdf

[2] https://eris.leipzig.de/appl/laura/wp5/kais02.nsf/docid/2BE5B12C52054BBFC1257CED002D822C/$FILE/V-rb-2072-ausfertigung.pdf

In [ ]:
years = [2020,2030,2040]
goals = pd.DataFrame(
    index = years,
    data = {
        "population": population.loc[years].value,
        "per_capita": [4.26, 3.48, 2.93]
    }
)

goals['value'] = goals.population * goals.per_capita / 1000
goals['category'] = "Gesamt"
goals['type'] = "geplant"
goals['note'] = "Klimaschutzkonzept 2014-2020"
goals

## Emissionen 1990

Als Referenzjahr wird oft das Jahr 1990 verwendet, so auch bei Klimawatch. Die Daten für Leipzig finden sind auch im Klimaschutzkonzept, und wir fügen sie hier den historischen Emissionen hinzu.

In [ ]:
emissions_1990 = pd.DataFrame([
    {'year': 1990, 'category': 'Wirtschaft und kommunale Einrichtungen', 'per_capita': 4.83},
    {'year': 1990, 'category': 'Haushalte', 'per_capita': 4.79},
    {'year': 1990, 'category': 'Verkehr', 'per_capita': 1.69},
    {'year': 1990, 'category': 'Gesamt', 'per_capita': 11.31}
])
emissions_1990.set_index('year', inplace=True)

population_1990 = 557300
emissions_1990['value'] = emissions_1990['per_capita'] * population_1990 / 1000
emissions_1990['type'] = 'real'
emissions_1990['note'] = 'Klimaschutzkonzept 2014-2020'

emissions_1990

## Output

Damit haben wir alle Daten zusammen, die für Klimawatch benötigt werden. Im Folgenden werden die Daten in die Form gebracht wie sie in der Anleitung (1) auf der Webseite beschrieben ist. 

[1] https://klimawatch.codefor.de/anleitung/


In [ ]:
columns = ['category', 'type', 'value', 'note']
consolidated = pd.DataFrame(columns=columns)

# emission data
consolidated = consolidated.append(emissions_1990[columns])
consolidated = consolidated.append(emissions[columns])

# emission goals
consolidated = consolidated.append(goals[columns])

# latest population data
consolidated = consolidated.append(pd.DataFrame(
    {
        'category': 'Einwohner',
        'type': 'Einwohner',
        'value': population.loc[2018, 'value'],
        'note': 'Bevölkerungsstatistik Leipzig'
    },
    index = [2018]
))

# add header for year
consolidated.index.name = 'year'

consolidated

In [ ]:
consolidated.to_csv('../leipzig.csv')